# Measuring US Industry Level Productivity for 1947-2023
## [Juan Ignacio Vizcaino](https://www.jivizcaino.com/) and [Selim Elbadri](https://www.selimelbadri.com/) 

We here combine data from US KLEMS, March 2017 Release, with BEA-BLS Integrated Industry-Level Production Account for 1947–2016, and BEA-BLS Integrated Industry-Level Production Account for 1997-2023 to produce industry-level measures of Gross Output (GO), Value Added (VA), Capital (CAP), Labor (LAB), and Intermediate Inputs (II), in Nominal and Real terms. We also provide Quantity Indices for GO, VA, CAP, LAB, and II, and utilize these indices to compute measures of Total Labor Productivity (LP) and Total Factor Productivity (TFP) for the corresponding time period, following the methodology in US KLEMS, April 2013 Release.
See our [Gihub repo](https://github.com/selbadri/Measuring-US-Industry-Level-Productivity-1947-to-2023) for details on data processing. 

## 1. Preliminaries

#### 1.1 Set up the Working Directory

In [1]:
import_file_path = rf"..\\Input"
export_file_path = rf"..\\Output"

#### 1.2. Requirements: Import Required Libraries

In [2]:
import pandas as pd
import numpy as np
import os
from functools import reduce

#### 1.3. Define Frequently Used Functions and Lists

Define some widely used functions (see docstring for details on what these functions do).

In [3]:
def tornqvist_index(df, q_vars, v_vars, industry_column=None):
    """
    This function calculatest a Tornqvist quantity index.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing quantity and value variables
    q_vars : list
        List of column names representing quantity variables to be aggregated
    v_vars : list  
        List of column names representing corresponding value variables
        used for weighting. Must be same length as q_vars.
    industry_column : str, optional
        Column name for industry/group identifier. If provided, calculations
        are performed separately for each industry group. If None, treats
        entire dataset as single time series.
        
    Returns
    -------
    pandas.Series
        Tornqvist quantity index values, normalized to 100 for the first
        observation of each group (or first row if no grouping)
        
    Notes
    -----
    The Tornqvist index formula used is:
    ln(QI_t/QI_{t-1}) = Σ [0.5 * (w_i,t + w_i,t-1) * ln(q_i,t/q_i,t-1)]
    
    Where:
    - QI_t is the quantity index at time t
    - w_i,t is the value share of component i at time t (v_i,t / Σv_j,t)
    - q_i,t is the quantity of component i at time t
    
    The index is initialized to 100 for the first period and cumulative
    products are calculated to generate the full time series.
    
    Examples
    --------
    >>> # Calculate aggregate capital quantity index
    >>> capital_qi = tornqvist_index(
    ...     df, 
    ...     q_vars=['it_quantity', 'structures_quantity', 'equipment_quantity'],
    ...     v_vars=['it_value', 'structures_value', 'equipment_value'],
    ...     industry_column='industry_id'
    ... )
    """
    
    df = df.copy()
    total_v = df[v_vars].sum(axis=1)
    for col in v_vars:
        df[f'w_{col}'] = df[col] / total_v
        
    log_index_sum = 0
    for q_var, v_var in zip(q_vars, v_vars):
        w_var = f'w_{v_var}'
        
        if industry_column is not None:
            log_change = np.log(df[q_var] / df.groupby(industry_column)[q_var].shift(1))
            avg_weight = 0.5 * (df[w_var] + df.groupby(industry_column)[w_var].shift(1))
        else:
            log_change = np.log(df[q_var] / df[q_var].shift(1))
            avg_weight = 0.5 * (df[w_var] + df[w_var].shift(1))

        log_index_sum += avg_weight * log_change

    q_growth_rate = np.exp(log_index_sum)

    if industry_column is not None:
        q_growth_rate.loc[df.groupby(industry_column).head(1).index] = 100
        qi = q_growth_rate.groupby(df[industry_column]).cumprod()
    else:
        q_growth_rate.iloc[0] = 100
        qi = q_growth_rate.cumprod()

    return qi

In [4]:
def normalise(df, variable, year, industry_column):
    """Normalize variable to base year = 100 by industry."""
    base_values = df[df['year'] == year].set_index(industry_column)[variable]
    normaliser = df[industry_column].map(base_values)
    df[variable] = (df[variable] / normaliser) * 100
    return df[variable]

In [5]:
def clean(obj):
    """
    Standardize names by replacing spaces with underscores and converting to lowercase.
    This function is used for making column names consistent across different data sources.
    
    Parameters
    ----------
    obj : pd.DataFrame, list, or str
        Object to clean
        
    Returns
    -------
    Same type as input with standardized naming
    """
    if isinstance(obj, pd.DataFrame):
        obj.columns = [col.replace(' ', '_').lower() for col in obj.columns]
        return obj
    elif isinstance(obj, list):
        return [s.replace(' ', '_').lower() for s in obj]
    elif isinstance(obj, str):
        return obj.replace(' ', '_').lower()

In [6]:
def aggregate_industries(df):
    """
    Create industry aggregates by combining individual industries into larger sectoral groups.
    
    This function is used to produce consistent industry aggregates from 1947-2023, given that the
    BEA-BLS Industry Production Account Experimental file has aggregated data for industries 29-36,
    37-40, 41-44, 47-49, 51-52, 54-56, and 57-58.
    
    The function creates aggregate industry groups (e.g., combining industries 29-36 into group 2936) for 1947-2023 
    and computes appropriate Tornqvist quantity indices that maintain economic consistency.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe with MultiIndex ['year', 'industry_id'] containing individual
        industry data with quantity indices and value added measures.
        Required columns: qi_variables (quantity indices) and 'VA' (value added)
        
    Returns
    -------
    pandas.DataFrame
        Original dataframe with additional rows for aggregate industries, each containing
        appropriately calculated Tornqvist quantity indices for the combined sectors.
        MultiIndex structure ['year', 'industry_id'] is preserved.
        
    Notes
    -----
    The function operates by:
    1. Creating industry groups according to aggregate_groups mapping (see list below)
    2. For each group and each quantity index variable:
       - Extracting component industry data (quantities and values)
       - Applying Tornqvist aggregation formula using individual industry weights
       - Assigning new aggregate industry code to results
    3. Concatenating aggregate results with original individual industry data
    
    The Tornqvist aggregation ensures that the aggregate indices properly reflect
    the relative economic importance of component industries through value-based weighting.
    
    Dependencies
    ------------
    - Requires global variables: qi_variables, aggregate_groups
    - Uses tornqvist_index() function for proper economic aggregation
    - Assumes df contains 'VA' (Value Added) column for weighting
    
    Examples
    --------
    Typical usage in productivity analysis pipeline:
    >>> # After calculating individual industry quantity indices
    >>> df_with_aggregates = aggregate_industries(df_individual)
    >>> # Result contains both individual industries (1, 2, 3, ...) 
    >>> # and aggregates (2936, 3740, 4144, ...)
    """
    aggregate_dict = {}
    df = df.reset_index()
    for qi in qi_variables:
        for agg_code, industries in aggregate_groups.items():
            data_slice = df[df['industry_id'].isin(industries)].copy()

            q_vars = []
            v_vars = []

            for industry in industries:
                q = f'{industry}_{qi}'
                v = f'{industry}_va'

                services_index = df[df['industry_id'] == industry][['year', qi]].rename(columns={qi: q})
                va = df[df['industry_id'] == industry][['year', 'VA']].rename(columns={'VA': v})
            
                data_slice = data_slice.merge(services_index, on='year', how='left')
                data_slice = data_slice.merge(va, on='year', how='left')
            
                q_vars.append(q)
                v_vars.append(v)

            sum_cols = ['VA']
            data_slice[sum_cols] = data_slice.groupby('year')[sum_cols].transform('sum')
            data_slice = data_slice.drop_duplicates(subset='year')

            data_slice[qi] = tornqvist_index(data_slice, q_vars=q_vars, v_vars=v_vars, industry_column='industry_id')
            data_slice['industry_id'] = agg_code

            if agg_code not in aggregate_dict:
                aggregate_dict[agg_code] = data_slice[['year', 'industry_id'] + [qi] + sum_cols]
            else:
                aggregate_dict[agg_code] = aggregate_dict[agg_code].merge(data_slice[['year', 'industry_id', qi]], on=['year', 'industry_id'], how='left')

    aggregate_df = pd.concat(aggregate_dict.values(), ignore_index=True)

    df = pd.concat([df, aggregate_df], ignore_index=True).set_index(['year', 'industry_id'])
    return df

In [7]:
def chain(df_1, df_2, year):
    """
    Chain together two time series datasets with overlapping periods using index linking.
    
    This function is used for creating consistent long-term economic time series 
    when combining data from different sources or periods. It ensures continuity by
    scaling the second dataset to match the level of the first dataset at the 
    overlapping year, maintaining growth rates while eliminating level discontinuities.
    
    Parameters
    ----------
    df_1 : pandas.DataFrame
        First (earlier) dataset with MultiIndex ['year', 'industry_id'].
        Contains quantity index variables that serve as the reference level.
    df_2 : pandas.DataFrame
        Second (later) dataset with MultiIndex ['year', 'industry_id'].
        Will be scaled to match df_1 levels at the linking year.
    year : int
        Overlapping year used for linking the two datasets. This year's values
        from df_1 provide the scaling factors for df_2.
        
    Returns
    -------
    pandas.DataFrame
        Combined dataset with MultiIndex ['year', 'industry_id'] containing
        the chained time series. df_1 data (excluding linking year) plus
        df_2 data scaled to maintain continuity.
        
    Notes
    -----
    The chaining process:
    1. Extracts scaling factors from df_1 at the linking year
    2. Applies these factors to all df_2 observations to maintain relative levels
    3. Removes the linking year from df_1 to avoid duplication
    4. Combines the datasets preserving chronological order
    
    The scaling formula for each quantity index (qi) is:
    df_2_scaled[qi] = df_2[qi] * (df_1[qi, linking_year] / 100)
    
    This preserves growth rates in df_2 while ensuring level consistency with df_1.
    
    Dependencies
    ------------
    - Requires global variable: qi_variables (list of quantity index column names)
    - Both datasets must have matching industry_id values for proper scaling
    - Missing scaling factors are filled with 100 (no adjustment)
    
    Examples
    --------
    Typical usage in productivity analysis:
    >>> # Chain 1947-1996 data with 1997-2023 data using 1997 as link year
    >>> combined_data = chain(early_period_df, later_period_df, 1997)
    >>> # Result: consistent time series from 1947-2023
    """
    df_1 = df_1.reset_index()
    df_2 = df_2.reset_index()

    df_2_scaled = df_2.copy()

    for qi in qi_variables:
        scalers = f'{qi}_scalers'

        df_1_year = df_1[df_1['year'] == year][['industry_id', qi]].rename(columns={qi: scalers})

        df_2_scaled = df_2_scaled.merge(df_1_year, on='industry_id', how='left')
        df_2_scaled[scalers] = df_2_scaled[scalers].fillna(100)
        df_2_scaled[qi] *= df_2_scaled[scalers]
        df_2_scaled[qi] = df_2_scaled[qi] / 100
        df_2_scaled = df_2_scaled.drop(columns=scalers)

    df_1 = df_1[df_1['year'] != year]

    df_new = pd.concat([df_1, df_2_scaled], ignore_index=True)
    df_new = df_new.set_index(['year', 'industry_id']).sort_values(by=['industry_id', 'year'])

    return df_new

In [8]:
def constant_values(df, variable, year):
    """
    Convert nominal values to real (constant dollar) values using implicit price deflator.
    
    This function implements the standard economic method for converting nominal time series 
    into real values by extracting implicit price indices and deflating current values.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing nominal values and corresponding quantity indices.
        Must include columns: 'year', 'industry_id', variable, and f'{variable}_QI'
    variable : str
        Name of the nominal variable to convert (e.g., 'GO', 'CAP', 'LAB', 'II').
        Must have a corresponding quantity index column f'{variable}_QI'
    year : int
        Base year for the constant dollar calculation. Real values will be expressed
        in this year's dollars (e.g., 2009 for "2009 constant dollars")
        
    Returns
    -------
    pandas.Series
        Real (constant dollar) values for the specified variable, rounded to 4 decimals.
        Series name will be f'REAL_{variable}' (e.g., 'REAL_GO')
        
    Notes
    -----
    The conversion process follows these steps:
    
    1. Create value index:           V_index(t) = V(t) / V(base_year) × 100
    2. Extract implicit price index: P_index(t) = V_index(t) / Q_index(t)  
    3. Calculate real values:        REAL_V(t)  = V(t) / P_index(t)
    
    Where:
    - V(t)       = nominal value at time t
    - Q_index(t) = quantity index at time t  
    - P_index(t) = implicit price index at time t
    
    This method ensures that real values reflect only quantity changes, with price
    effects removed through the implicit deflation process.
    
    
    Examples
    --------
    Convert nominal gross output to 2009 constant dollars:
    >>> real_go = constant_values(df, 'GO', 2009)
    >>> # Result: REAL_GO series in 2009 constant dollars
    
    Convert nominal capital to 1997 constant dollars:  
    >>> real_cap = constant_values(df, 'CAP', 1997)
    >>> # Result: REAL_CAP series in 1997 constant dollars
    """
    
    df = df.copy()
    values_year = {}
    df_year = df[df['year'] == year].set_index('industry_id')
    values_year = df_year[variable].to_dict()
    df[f'{variable}_{year}'] = df['industry_id'].map(values_year)
    df[f'{variable}_value_index'] = df[variable] / df[f'{variable}_{year}']
    df[f'{variable}_value_index'] *= 100
    df[f'{variable}_price_index'] = df[f'{variable}_value_index'] / df[f'{variable}_QI']
    df[f'REAL_{variable}'] = df[variable] / df[f'{variable}_price_index']
    df[f'REAL_{variable}'] = df[f'REAL_{variable}'].round(4)
    return df[f'REAL_{variable}']

In [9]:
def recover_index(df, index_name, variable):
    """
    Reconstruct level indices from cumulative log differences (growth rates).
    
    This function converts log difference series (growth rates) back into index levels that 
    can be normalized and compared across time periods. It is used to produce chained quantity
    indices for when we have data on for overlapping periosds from different sources.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing log difference variables by industry.
        Must include 'industry_id' column and f'delta_ln_{variable}' column.
    index_name : str
        Name for the recovered index variable (e.g., 'TFP_VA', 'LP_GO', 'VA_QI').
        This becomes the column name for the output index.
    variable : str
        Base name of the log difference variable to recover from.
        Function expects column f'delta_ln_{variable}' to exist in df.
        
    Returns
    -------
    pandas.Series
        Recovered index series normalized to 100 for base period, with name=index_name.
        Index values represent cumulative changes from the first observation.
        
    Notes
    -----
    Mathematical Process:
    1. Cumulative sum: ln(Index_t) = Σ(Δln(Index_s)) for s = 1 to t
    2. Exponentiation: Index_t     = exp(ln(Index_t))  
    3. Normalization:  Index_t     = Index_t × 100
    
    Where Δln(Index_t) represents the log difference (growth rate) at time t.
    
    The cumulative sum is performed within each industry group, ensuring that
    each industry's index starts from its own baseline period.
    
    Examples
    --------
    Recover Total Factor Productivity index from growth rates:
    >>> tfp_index = recover_index(df, 'TFP_VA', 'TFP_VA')
    >>> # Creates index from delta_ln_TFP_VA growth rates
    
    Recover Value Added quantity index:
    >>> va_qi = recover_index(df, 'VA_QI', 'VA_QI')  
    >>> # Creates index from delta_ln_VA_QI growth rates
    """
    
    df['ln_' + variable] = df.groupby('industry_id')['delta_ln_' + variable].cumsum().fillna(0)
    df[index_name] = np.exp(df['ln_' + variable])
    df[index_name] *= 100
    return df[index_name]

In [10]:
def lag(df, variable):
    """
    Create lagged (previous period) values of a variable by industry group.
    
    It creates one-period lags within each industry group, ensuring that lagged values 
    don't bleed across different industries in the panel data.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing the variable to lag. Must include 'industry_id' column
        for proper grouping and should be sorted by time within each industry.
    variable : str
        Name of the column to create lagged values for. The lagged column will be named
        f'{variable}_lag' (e.g., 'VA/GO' becomes 'VA/GO_lag').
        
    Returns
    -------
    pandas.Series
        Lagged values of the specified variable, with NaN for the first observation
        of each industry group (no previous period available).
        
    Examples
    --------
    Create lagged value-added share for Tornqvist calculations:
    >>> va_go_lag = lag(df, 'VA/GO')
    >>> # Used in: avg_share = 0.5 * (df['VA/GO'] + va_go_lag)
    
    Create lagged labor share:
    >>> lab_va_lag = lag(df, 'LAB/VA') 
    >>> # Used in TFP calculations for proper factor weighting
    """
    df[f'{variable}_lag'] = df.groupby('industry_id')[variable].transform(lambda x: x.shift(1))
    return df[f'{variable}_lag']

In [11]:
def delta(df, variable):
    """
    Calculate first differences (period-over-period changes) of a variable by industry group.
    This function computes simple arithmetic differences between consecutive periods
    within each industry.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing the variable to difference. Must include 'industry_id' 
        column for proper grouping and should be sorted by time within each industry.
    variable : str
        Name of the column to calculate differences for. The difference column will be 
        named f'delta_{variable}' (e.g., 'ln_REAL_GO' becomes 'delta_ln_REAL_GO').
        
    Returns
    -------
    pandas.Series
        First differences of the specified variable, with NaN for the first observation
        of each industry group (no previous period for comparison).
        
    Notes
    -----
    Mathematical Operation:
    delta_X(t) = X(t) - X(t-1)
    
    When applied to logarithmic variables:
    delta_ln_X(t) = ln(X(t)) - ln(X(t-1)) = ln(X(t)/X(t-1))
    
    This represents the continuous growth rate, which is approximately equal to
    the percentage change for small changes.
    
    Examples
    --------
    Calculate log growth rate of real gross output:
    >>> go_growth = delta(df, 'ln_REAL_GO')
    >>> # Result: delta_ln_REAL_GO = ln(GO_t) - ln(GO_{t-1})
    
    Calculate change in labor share:
    >>> share_change = delta(df, 'LAB/VA') 
    >>> # Result: delta_LAB/VA = (LAB/VA)_t - (LAB/VA)_{t-1}
    """
    df[f'delta_{variable}'] = df.groupby('industry_id')[variable].transform(lambda x: x - x.shift(1))
    return df[f'delta_{variable}']

Define two widely used lists. The first list *aggregate_groups* is useful to aggregate industries at the same level as the BEA-BLS Industry Production Account Experimental for the period of 1947-1963.
The second set of lists define variable groupings that are useful for computations later on.

In [12]:
aggregate_groups = {
    2936: list(range(29, 37)),
    3740: list(range(37, 41)),
    4144: list(range(41, 45)),
    4749: list(range(47, 50)),
    5152: list(range(51, 53)),
    5456: list(range(54, 57)),
    5758: list(range(57, 59))}

In [13]:
core_variables               = ['GO', 'CAP', 'LAB', 'II']
constant_variables           = ['REAL_GO', 'REAL_CAP', 'REAL_LAB', 'REAL_II']
qi_variables                 = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI']
productivity_index_variables = ['TFP_GO', 'TFP_VA', 'LP_GO', 'LP_VA']

## 2. Merge the BEA-BLS data with US KLEMS 2017
We combine data from US KLEMS (March 2017 Release) with BEA-BLS Integrated Industry-Level Production Account for 1947–2016, and BEA-BLS Integrated Industry-Level Production Account for 1997-2023.
This produces two datsets containing GO, VA, CAP, LAB, II in nominal termns, real terms, as well as quantity indices, by industry for 44 industries between 1947 and 2023 and 63 industries for 1963-2016.

##### 2.1 Merge the two periods available for the *industry-production-account-experimental* dataset. 
This requires aggregating industries consistently first

In [14]:
required_columns = ['yr', 'indnum', 'go.', 'goqi.', 'ii.', 'iiqi.', 'vlcol.', 'vln.', 'vkit.', 'vksoft.', 'vkRD.', 'vkart.', 'vkoth.', 'qlindexcol_merge.', 'qlindexn_merge.', 'qkit.', 'qks.', 'qkrd.', 'qka.', 'qko.', 'hrs']

data_1 = pd.read_excel(os.path.join(import_file_path,'industry-production-account-experimental.xlsx'), sheet_name='1947-1963', skiprows=1, usecols=required_columns)
data_2 = pd.read_excel(os.path.join(import_file_path,'industry-production-account-experimental.xlsx'), sheet_name='1963-2016', skiprows=1, usecols=required_columns)

Build quantity indices

In [15]:
data_1 = data_1.reset_index()
data_2 = data_2.reset_index()

q_vars_dict_1 = {
    'GO':   ['goqi.'],
    'CAP':  ['qkit.', 'qks.', 'qkrd.', 'qka.', 'qko.'],
    'LAB':  ['qlindexcol_merge.', 'qlindexn_merge.'],
    'II':   ['iiqi.']}

v_vars_dict_1 = {
    'GO':   ['go.'],
    'CAP':  ['vkit.', 'vksoft.', 'vkRD.', 'vkart.', 'vkoth.'],
    'LAB':  ['vlcol.', 'vln.'],
    'II':   ['ii.']}

for df in [data_1, data_2]:
    for variable in core_variables:
        q_vars = q_vars_dict_1[variable]
        v_vars = v_vars_dict_1[variable]
        df[f'{variable}_QI'] = tornqvist_index(df, q_vars, v_vars, industry_column='indnum')

data_1['VA'] = data_1['go.'] - data_1['ii.']
data_2['VA'] = data_2['go.'] - data_2['ii.']

data_1.reset_index(inplace=True)
data_2.reset_index(inplace=True)
data_1 = data_1[qi_variables + ['hrs', 'VA', 'indnum', 'yr']]
data_2 = data_2[qi_variables + ['hrs', 'VA', 'indnum', 'yr']]
data_1 = data_1.rename(columns={'hrs': 'TOT_HRS', 'indnum': 'industry_id', 'yr': 'year'})
data_2 = data_2.rename(columns={'hrs': 'TOT_HRS', 'indnum': 'industry_id', 'yr': 'year'})
data_1 = data_1.set_index(['year', 'industry_id'])
data_2 = data_2.set_index(['year', 'industry_id'])

Aggregate Industries following the BEA-BLS *industry-production-account-experimental* aggregation for 1947-1963.

In [16]:
qi_variables = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI', 'TOT_HRS']

data_2 = aggregate_industries(data_2)
data_2 = data_2.drop(columns=['VA'])
data_1 = data_1.drop(columns=['VA'])

In [17]:
data_1.reset_index(inplace=True)
data_2.reset_index(inplace=True)

data_1['TOT_HRS']   = normalise(data_1, 'TOT_HRS', year=1947, industry_column='industry_id')
data_2['TOT_HRS']   = normalise(data_2, 'TOT_HRS', year=1963, industry_column='industry_id')

In [18]:
df_qi_47_to_16    = chain(data_1, data_2, 1963)
df_47_to_16_hours = df_qi_47_to_16['TOT_HRS'].copy()
qi_variables      = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI']

Chain together the two subperiods.

##### 2.2 Merge the BEA data (1997 to 2023) with KLEMS2017 (1947 to 2014)

In [19]:
cap_qty_list  = ['Capital_Art_Quantity', 'Capital_R&D_Quantity', 'Capital_IT_Quantity', 'Capital_Other_Quantity', 'Capital_Software_Quantity']
cap_comp_list = ['Capital_Art Compensation', 'Capital_R&D Compensation', 'Capital_IT Compensation', 'Capital_Other Compensation', 'Capital_Software Compensation']

lab_qty_list  = ['Labor_Col_Quantity', 'Labor_NoCol_Quantity']
lab_comp_list = ['Labor_Col Compensation', 'Labor_NoCol Compensation']

ii_qty_list   = ['Energy_Quantity', 'Materials_Quantity', 'Services_Quantity']
ii_comp_list  = ['Energy Compensation', 'Materials Compensation', 'Service Compensation']

relevant_sheets = cap_qty_list + cap_comp_list + lab_qty_list + lab_comp_list + ii_qty_list + ii_comp_list + ['Value Added'] + ['VA_Quantity'] + ['Gross Output'] + ['Gross Output_Quantity'] + ['Labor Hours_Quantity']

long_data = []

for sheet in relevant_sheets:
    df = pd.read_excel(os.path.join(import_file_path, 'industry-production-account-capital.xlsx'), sheet_name=sheet, header=[1])
    df = df.dropna(how='all')
    df.rename(columns={df.columns[0]: 'industry'}, inplace=True) 
    df_long = df.melt(id_vars='industry', var_name='Year', value_name=sheet)
    long_data.append(df_long)

df = reduce(lambda left, right: pd.merge(left, right, on=['industry', 'Year'], how='outer'), long_data)

industry_order    = long_data[0]['industry'].drop_duplicates().tolist()
industry_id_map   = {industry: i+1 for i, industry in enumerate(industry_order)}
df['industry_id'] = df['industry'].map(industry_id_map)

df = clean(df)
cap_qty_list  = clean(cap_qty_list)
cap_comp_list = clean(cap_comp_list)
lab_qty_list  = clean(lab_qty_list)
lab_comp_list = clean(lab_comp_list)
ii_qty_list   = clean(ii_qty_list)
ii_comp_list  = clean(ii_comp_list)

df['year'] = df['year'].astype(int)
df = df.set_index(['industry_id', 'year']).sort_index(level=['industry_id'])

In [20]:
df_klems = pd.read_excel(os.path.join(import_file_path, 'usa_wk_mar_2017.xlsx'), sheet_name='KLEMdata', header=[1])
df_klems = df_klems[['year', 'industry', 'gross output', 'capital', 'labor', 'intermediate']]
df_klems = df_klems.rename(columns={'gross output': 'GO', 'capital': 'CAP', 'labor': 'LAB', 'intermediate': 'II', 'industry': 'industry_id'})

Import and process KLEMS data

In [21]:
df = df.rename(columns={'gross_output': 'GO', 'value_added': 'VA', 'labor_hours_quantity': 'TOT_HRS'})
df['LAB'] = df[lab_comp_list].sum(axis=1)
df['CAP'] = df[cap_comp_list].sum(axis=1)
df['II']  = df[ii_comp_list].sum(axis=1)

df_post_2014 = df.copy()
df_post_2014 = df_post_2014.reset_index()
df_post_2014 = df_post_2014[core_variables + ['industry_id', 'year', 'TOT_HRS']]
df_post_2014 = df_post_2014[df_post_2014['year'] > 2014]

df_klems_62_63 = df_klems['industry_id'].isin([62, 63])
df_62 = df_klems[df_klems_62_63].groupby('year').sum()
df_62['industry_id'] = 62
df_62 = df_62.reset_index()

df_klems_64_65 = df_klems['industry_id'].isin([64, 65])
df_63 = df_klems[df_klems_64_65].groupby('year').sum()
df_63['industry_id'] = 63
df_63 = df_63.reset_index()

df_klems = df_klems[~df_klems['industry_id'].isin([62, 63, 64, 65])]
df_klems = pd.concat([df_klems, df_62, df_63], ignore_index=True)

df_nominal_47_to_23 = pd.concat([df_post_2014, df_klems], ignore_index=True)

# nominal values in aggregate industries

aggregate_dict = {}

for agg_code, industries in aggregate_groups.items():
    data_slice = df_nominal_47_to_23[df_nominal_47_to_23['industry_id'].isin(industries)].copy()
    data_slice[core_variables] = data_slice.groupby('year')[core_variables].transform('sum')
    data_slice = data_slice.drop_duplicates(subset='year')
    data_slice['industry_id'] = agg_code
    aggregate_dict[agg_code] = data_slice[['year', 'industry_id'] + core_variables]

df_nominal_47_to_23 = pd.concat([df_nominal_47_to_23] + list(aggregate_dict.values()), ignore_index=True).set_index(['year', 'industry_id']).sort_index(level=['industry_id'])

Nominal values first.

In [22]:
df = df.reset_index()

q_vars_dict_2 = {
    'GO': ['gross_output_quantity'],
    'CAP': cap_qty_list,
    'LAB': lab_qty_list,
    'II':  ii_qty_list}

v_vars_dict_2 = {
    'GO':  ['GO'],
    'CAP': cap_comp_list,
    'LAB': lab_comp_list,
    'II':  ii_comp_list}

for variable in core_variables:
    q_vars = q_vars_dict_2[variable]
    v_vars = v_vars_dict_2[variable]
    df[f'{variable}_QI'] = tornqvist_index(df, q_vars, v_vars, industry_column='industry_id')

df['year'] = df['year'].astype(int)

df_qi_97_to_23 = df[qi_variables + ['year', 'industry_id', 'VA', 'TOT_HRS']].copy()
df_qi_97_to_23['TOT_HRS'] = normalise(df, 'TOT_HRS', year=1997, industry_column='industry_id')
df_qi_97_to_23 = df_qi_97_to_23.set_index(['year', 'industry_id']).sort_index(level=['industry_id'])

Quantity indices second.

In [23]:
qi_variables      = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI', 'TOT_HRS']
df_qi_97_to_23    = aggregate_industries(df_qi_97_to_23)
qi_variables      = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI']
df_qi_97_to_23    = df_qi_97_to_23.drop(columns=['VA'])
df_97_to_23_hours = df_qi_97_to_23['TOT_HRS'].copy()

#### 2.3 Chaining the data for 1947-2016 with 1997-2023

In [24]:
df_47_to_96_hours = df_47_to_16_hours[df_47_to_16_hours.index.get_level_values('year').astype(int) <= 1997]
qi_variables      = ['TOT_HRS']
df_47_to_23_hours = chain(df_47_to_96_hours, df_97_to_23_hours, 1997)
qi_variables      = ['GO_QI', 'CAP_QI', 'LAB_QI', 'II_QI']

In [25]:
df_qi_47_to_96 = df_qi_47_to_16[df_qi_47_to_16.index.get_level_values('year').astype(int) <= 1997]
df_qi_47_to_23 = chain(df_qi_47_to_96, df_qi_97_to_23, 1997)
df_qi_47_to_23 = df_qi_47_to_23.reset_index()

df_qi_47_to_23['year'] = df_qi_47_to_23['year'].astype(int)

for qi in qi_variables:
   df_qi_47_to_23[qi] = normalise(df_qi_47_to_23, qi, year=2009, industry_column='industry_id')

df_qi_47_to_23 = df_qi_47_to_23.set_index(['year', 'industry_id']).sort_index(level=['industry_id'])
df_47_to_23    = pd.merge(df_nominal_47_to_23[core_variables], df_qi_47_to_23[qi_variables],  left_index=True, right_index=True, how='inner').round(4)

In [26]:
df_47_to_23    = pd.merge(df_47_to_23, df_47_to_23_hours, left_index=True, right_index=True, how='inner')

#### 2.4 Compute Variables in Real Terms

In [27]:
df_47_to_23 = df_47_to_23.reset_index()

for variable in core_variables:
    df_47_to_23[f'REAL_{variable}'] = constant_values(df_47_to_23, variable, 2009)

df_47_to_23 = df_47_to_23.set_index(['year', 'industry_id']).sort_index(level=['industry_id'])
df_47_to_23 = df_47_to_23[core_variables + constant_variables + qi_variables + ['TOT_HRS']]
df          = df_47_to_23.copy()

#### 2.4 Compute Value Added

This follows the standard approach
$$
P_{V A}(t) Q_{V A}(t) = P_Y (t) Q_Y (t) - P_{II} (t) Q_{II} (t),
$$
wher $VA$ is Value Added, $Y$ is Gross Output, and $II$ is Itermediate Inputs. $P_{j}$ and $Q_{j}$ represent prices and quantities for component $j$, respectively.

In [28]:
df['VA'] = df['GO'] - df['II']

## 3. Compute TFP and LP Indices

Compute the share of VA in GO.

In [29]:
df['VA/GO'] = df['VA'] / df['GO']

### 3.1 Build a VA Tornqvist Quantity Index

To compute a value added quantity index, we start from the definition of a Tornqvist Quantity Index for total output $Y$

$$
\Delta \ln Q_Y (t) = \bar{\nu}_{V A} (t) \Delta \ln Q_{VA}(t)   + \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t),
$$

where

$$
\Delta \ln Q_{j}(t) = \ln Q_{j}(t) - \ln Q_{j}(t-1),
$$

and

$$
\bar{\nu}_{j}(t) = 0.5 \times \left( \frac{P_{j}(t) Q_{j}(t)}{P_Y (t) Q_Y (t)} + \frac{P_{j}(t - 1) Q_{j}(t - 1)}{P_Y (t - 1) Q_Y (t - 1)} \right)
$$

representing the Tornqvist weight for the component $j \in \left\lbrace VA,II \right\rbrace$ of $GO$.

Re arranging terms, we get:

$$
\Delta \ln Q_{VA}(t) = \frac{\Delta \ln Q_Y (t) - \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t)}{\bar{\nu}_{V A} (t)}
$$

Compute log differences

In [30]:
df['ln_REAL_GO'] = np.log(df['REAL_GO'])
df['ln_REAL_II'] = np.log(df['REAL_II'])

df['delta_ln_REAL_GO'] = delta(df, 'ln_REAL_GO')
df['delta_ln_REAL_II'] = delta(df, 'ln_REAL_II')

Compute the Tornqvist weights

In [31]:
df['II/GO'] = df['II'] / df['GO']

df['VA/GO_lag'] = lag(df, 'VA/GO')
df['II/GO_lag'] = lag(df, 'II/GO')

df['VA_tornqvist_GO_share'] = 0.5 * (df['VA/GO'] + df['VA/GO_lag'])
df['II_tornqvist_GO_share'] = 0.5 * (df['II/GO'] + df['II/GO_lag'])

Compute the log change of the Value Added quantity index

In [32]:
df['delta_ln_VA_QI'] = ((df['delta_ln_REAL_GO'] - (df['II_tornqvist_GO_share']*df['delta_ln_REAL_II']))/df['VA_tornqvist_GO_share'])

### 3.2 Build Labor Productivity Indices (LP)

We follow KLEMS and use total hours as a measure of Labor Input. Therefore:

* LP_VA:
$$
\Delta \ln LP_{VA}(t) = \Delta \ln Q_{VA}(t) - \Delta \ln Q_L(t)
$$

* LP_GO:
$$
\Delta \ln LP_{GO}(t) = \Delta \ln Q_{GO}(t) - \Delta \ln Q_L(t)
$$

In [33]:
df['ln_REAL_LAB']       = np.log(df['REAL_LAB'])
df['delta_ln_REAL_LAB'] = delta(df, 'ln_REAL_LAB')

In [34]:
df['ln_hours']       = np.log(df['TOT_HRS'])
df['delta_ln_hours'] = delta(df, 'ln_hours')

In [35]:
df['delta_ln_LP_VA'] = (df['delta_ln_VA_QI'] - df['delta_ln_hours'])
df['delta_ln_LP_GO'] = (df['delta_ln_REAL_GO'] - df['delta_ln_hours'])

### 3.3 Build Total Factor Productivity (TFP) Indices

Start with the Tornqvist Index for VA
$$
\Delta \ln Q_{VA}(t) = \Delta \ln TFP(t) + \bar{\psi}_L(t) \Delta \ln Q_L(t) + \bar{\psi}_K(t) \Delta \ln Q_K(t).
$$

Re-arranging terms, we get
$$
\Delta \ln TFP(t) = \Delta \ln Q_{VA}(t) - \bar{\psi}_L(t) \Delta \ln Q_L(t) - \bar{\psi}_K(t) \Delta \ln Q_K(t),
$$

where

$$
\bar{\psi}_{j}(t) = 0.5 \times \left( \frac{P_{j}(t) Q_{j}(t)}{P_{VA}(t) Q_{VA}(t)} + \frac{P_{j}(t-1) Q_{j}(t-1)}{P_{VA}(t-1) Q_{VA}(t-1)} \right),
$$

for $j \in \left\lbrace LAB,CAP \right\rbrace$ and  are nominal labor LAB ($L$) or nominal CAP ($K$).

In [36]:
df['LAB/VA'] = df['LAB'] / df['VA']
df['CAP/VA'] = df['CAP'] / df['VA']
df['LAB/VA_lag'] = lag(df, 'LAB/VA')
df['CAP/VA_lag'] = lag(df, 'CAP/VA')
df['L_tornqvist_VA_share'] = 0.5 * (df['LAB/VA'] + df['LAB/VA_lag'])
df['CAP_tornqvist_VA_share'] = 0.5 * (df['CAP/VA'] + df['CAP/VA_lag'])

df['LAB/GO'] = df['LAB'] / df['GO']
df['CAP/GO'] = df['CAP'] / df['GO']
df['LAB/GO_lag'] = lag(df, 'LAB/GO')
df['CAP/GO_lag'] = lag(df, 'CAP/GO')
df['L_tornqvist_GO_share'] = 0.5 * (df['LAB/GO'] + df['LAB/GO_lag'])
df['CAP_tornqvist_GO_share'] = 0.5 * (df['CAP/GO'] + df['CAP/GO_lag'])

In [37]:
df['ln_REAL_CAP'] = np.log(df['REAL_CAP'])
df['delta_ln_REAL_CAP'] = delta(df, 'ln_REAL_CAP')

Compute TFP growth rate.

In [38]:
df['delta_ln_TFP_VA'] = (df['delta_ln_VA_QI'] - (df['L_tornqvist_VA_share']*df['delta_ln_REAL_LAB']) - (df['CAP_tornqvist_VA_share']*df['delta_ln_REAL_CAP']))

In [39]:
df['delta_ln_TFP_GO'] = (df['delta_ln_REAL_GO'] - (df)['L_tornqvist_GO_share']*df['delta_ln_REAL_LAB'] - (df['CAP_tornqvist_GO_share']*df['delta_ln_REAL_CAP']) - (df['II_tornqvist_GO_share']*df['delta_ln_REAL_II']))

Compute the TFP, LP and VA indexes. Normalise to 2009

In [40]:
def recover_index(df, index_name):
    df['ln_' + index_name] = df.groupby('industry_id')['delta_ln_' + index_name].cumsum().fillna(0)
    df[index_name] = np.exp(df['ln_' + index_name])
    df[index_name] *= 100
    return df[index_name]

In [41]:
df = df.reset_index()

for variable in productivity_index_variables:
    df[variable] = recover_index(df, variable)
    df[variable] = normalise(df, variable, year=2009, industry_column='industry_id')
df['VA_QI'] = recover_index(df, 'VA_QI')
df['VA_QI'] = normalise(df, 'VA_QI', year=2009, industry_column='industry_id')

df = df.set_index(['year', 'industry_id']).sort_index(level=['industry_id'])

In [42]:
df = df[core_variables + ['VA'] + constant_variables + qi_variables + ['VA_QI'] + productivity_index_variables].round(4)
df_complete = df.copy()

# Apply different rounding to different variable groups
two_decimal_vars = ['GO', 'CAP', 'LAB', 'II', 'VA', 'REAL_GO', 'REAL_CAP', 'REAL_LAB', 'REAL_II']
df_complete[two_decimal_vars] = df_complete[two_decimal_vars].round(2)

# Round remaining variables to 4 decimal places
four_decimal_vars = [col for col in df_complete.columns if col not in two_decimal_vars]
df_complete[four_decimal_vars] = df_complete[four_decimal_vars].round(4)

In [43]:
df_complete.head()

,,GO,CAP,LAB,II,VA,REAL_GO,REAL_CAP,REAL_LAB,REAL_II,GO_QI,CAP_QI,LAB_QI,II_QI,VA_QI,TFP_GO,TFP_VA,LP_GO,LP_VA
year,industry_id,,,,,,,,,,,,,,,,,,
1947,1,31299.23,9744.23,10039.00,11516.0,19783.23,85420.74,58072.43,175614.44,71655.42,28.0331,91.8022,350.6423,37.4430,18.6973,34.2724,9.9743,4.7605,3.1752
1948,1,35560.70,11602.55,11443.14,12515.0,23045.70,93240.30,62451.03,169739.83,71305.97,30.5993,98.7240,338.9127,37.2604,21.4984,37.0196,11.2514,5.2976,3.7220
1949,1,29746.57,6988.22,11310.34,11448.0,18298.57,90564.31,62630.12,170840.07,70636.94,29.7211,99.0071,341.1095,36.9108,20.6429,35.9716,10.7512,5.2764,3.6647
1950,1,31972.91,9193.45,10284.46,12495.0,19477.91,95692.03,63096.65,158602.68,74804.06,31.4039,99.7446,316.6756,39.0883,21.7807,38.0812,11.7999,5.8552,4.0610
1951,1,37255.44,11954.30,10551.13,14750.0,22505.44,94865.65,67334.63,150626.21,79642.33,31.1327,106.4441,300.7493,41.6165,20.6164,36.6797,11.0925,6.0988,4.0387


## 4. Export the Data to an Excel Spreadsheet

In [44]:
industries_info = (
    pd.DataFrame(list(industry_id_map.items()), columns=['industry_name', 'industry_id'])
    .sort_values('industry_id')
    .reset_index(drop=True))

variables_dictionary = {
    '*': 'Note: variables with * are normalised to 2009 = 100',
    'GO': 'Nominal Gross Output',
    'CAP': 'Nominal Capital',
    'LAB': 'Nominal Labor',
    'II': 'Nominal Intermediate Inputs',
    'VA': 'Nominal Value Added',
    'REAL_GO*': 'Real Gross Output',
    'REAL_CAP*': 'Real Capital',
    'REAL_LAB*': 'Real Labor',
    'REAL_II*': 'Real Intermediate Inputs',
    'GO_QI*': 'Gross Output Quantity Index',
    'CAP_QI*': 'Capital Quantity Index',
    'LAB_QI*': 'Labor Quantity Index',
    'II_QI*': 'Intermediate Inputs Quantity Index',
    'VA_QI*': 'Value Added Quantity Index',
    'TFP_GO*': 'Total Factor Productivity Index (GO)',
    'TFP_VA*': 'Total Factor Productivity Index (VA)',
    'LP_GO*': 'Labor Productivity Index (GO)',
    'LP_VA*': 'Labor Productivity Index (VA)',
}

variables_info = pd.DataFrame(
    list(variables_dictionary.items()), 
    columns=['variable_name', 'variable_description'])

aggregate_industries_dictionary = {
    'Period': '1947-2023',
    '2936': 'Industries 29-36',
    '3740': 'Industries 37-40',
    '4144': 'Industries 41-44',
    '4749': 'Industries 47-49',
    '5152': 'Industries 51-52',
    '5456': 'Industries 54-56',
    '5758': 'Industries 57-58'
}

aggregate_industries_info = pd.DataFrame(
    list(aggregate_industries_dictionary.items()),
    columns=['industry_group', 'industry_ids']
)

# Create dataset for 1963-2023 with individual industries (1-63)
aggregated_industry_ids = [2936, 3740, 4144, 4749, 5152, 5456, 5758]
df_individual_industries = df_complete.copy()

# Filter out aggregated industries and keep individual industries 1-63
df_individual_industries = df_individual_industries.reset_index()
df_individual_industries = df_individual_industries[
    (~df_individual_industries['industry_id'].isin(aggregated_industry_ids)) & 
    (df_individual_industries['year'] >= 1963)
]

# Apply the same decimal formatting to the individual industries dataset
for col in two_decimal_vars:
    if col in df_individual_industries.columns:
        df_individual_industries[col] = df_individual_industries[col].apply(lambda x: float(f"{x:.2f}"))

for col in four_decimal_vars:
    if col in df_individual_industries.columns:
        df_individual_industries[col] = df_individual_industries[col].apply(lambda x: float(f"{x:.4f}"))

df_individual_industries = df_individual_industries.set_index(['year', 'industry_id']).sort_index(level=['industry_id'])

# Create dataset for 1947-2023 with properly organized aggregated industries
# Define which individual industries should be removed (they're part of aggregates)
individual_industries_to_remove = []
for industries_list in aggregate_groups.values():
    individual_industries_to_remove.extend(industries_list)

df_aggregated = df_complete.copy().reset_index()

# Remove individual industries that are part of aggregates
df_aggregated = df_aggregated[~df_aggregated['industry_id'].isin(individual_industries_to_remove)]

# Create a custom sorting order for industries
def get_sort_key(industry_id):
    """Create sort key to place aggregates in correct positions"""
    if industry_id <= 28:
        return industry_id
    elif industry_id == 2936:  # Should come after 28
        return 28.5
    elif industry_id <= 36:
        return industry_id
    elif industry_id == 3740:  # Should come after 36 (but we removed individual 29-36)
        return 36.5
    elif industry_id <= 40:
        return industry_id
    elif industry_id == 4144:  # Should come after 40
        return 40.5
    elif industry_id <= 46:
        return industry_id
    elif industry_id == 4749:  # Should come after 46 (we removed 47-49)
        return 46.5
    elif industry_id <= 50:
        return industry_id
    elif industry_id == 5152:  # Should come after 50
        return 50.5
    elif industry_id <= 53:
        return industry_id
    elif industry_id == 5456:  # Should come after 53
        return 53.5
    elif industry_id == 5758:  # Should come after 5456
        return 53.6
    else:
        return industry_id

# Apply custom sorting - by industry first, then by year
df_aggregated['sort_key'] = df_aggregated['industry_id'].apply(get_sort_key)
df_aggregated = df_aggregated.sort_values(['sort_key', 'year']).drop('sort_key', axis=1)

# Apply the same decimal formatting to the aggregated dataset
for col in two_decimal_vars:
    if col in df_aggregated.columns:
        df_aggregated[col] = df_aggregated[col].apply(lambda x: float(f"{x:.2f}"))

for col in four_decimal_vars:
    if col in df_aggregated.columns:
        df_aggregated[col] = df_aggregated[col].apply(lambda x: float(f"{x:.4f}"))

df_aggregated = df_aggregated.set_index(['year', 'industry_id'])

# Export both datasets to the same Excel file with separate sheets
file_path = os.path.join(export_file_path, "EV_production_accounts_1947to2023.xlsx")

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Info sheet
    industries_info.to_excel(writer, sheet_name='Info', index=False, startrow=1, startcol=1)
    aggregate_industries_info.to_excel(writer, sheet_name='Info', index=False, startrow=1, startcol=4)
    variables_info.to_excel(writer, sheet_name='Info', index=False, startrow=1, startcol=7)
    
    # Data sheets
    df_aggregated.to_excel(writer, sheet_name='Data_44Ind_1947-2023', index=True)
    df_individual_industries.to_excel(writer, sheet_name='Data_63Ind_1963-2023', index=True)